In [1]:
def mod_mul_standard(a, b, p):
    result = (a * b) % p
    if result > (p>>1):
        result -= p
    return result

def mod_add(a, b, p):
    result = (a + b)
    if result > (p>>1):
        result -= p
    return result

def mod_sub(a, b, p):
    result = (a - b) + p
    if result > (p>>1):
        result -= p
    return result


def check_arrays(a,b,p):
    if a == b:
        print("exactly equal")
        return True
    print("not exactly equal")

    if len(a) != len(b):
        print("lengths not equal")
        return False


    for i in range(len(a)):
        if (a[i] - b[i]) % p != 0:
            print("element", i, "not equal even in modulus class")
            return False
    print("But equal in modulus class")
    return True

    



In [2]:
def Mu(b,q):
    mu = 2*int(((b << 31)/(2*q))+0.5)
    return mu


def Barrett_mul(a, b, p, mu_b):
    # Perform the multiplication
    z = (a * b) % (1 << 32)
    if z >= (1 << 31):
        z -= (1 << 32)  # Convert to signed 32-bit integer

    # Perform the multiplication for mu_b and get the high 32 bits
    temp = (a * (mu_b << 1)) % (1 << 64)
    t_high = (temp >> 32) % (1 << 32)
    if t_high >= (1 << 31):
        t_high -= (1 << 32)  # Convert to signed 32-bit integer



    
    result = z - t_high * p
    # Simulate 32-bit signed integer overflow for the result
    result %= (1 << 32)
    if result >= (1 << 31):
        result -= (1 << 32)  # Convert to signed 32-bit integer

    return result

In [3]:
def CT_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    # Second layer
    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    return t0, t1, t2, t3

def GS_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    # Second layer
    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    return t0, t1, t2, t3


In [4]:
from data import *

logn_top = 10
n = 1 << logn_top
temp = [0] *  (8 * n)

temp[0:n] = f[0:n]
temp[n:(n<<1)] = g[0:n]

In [5]:
ptrf = 0
ptrg = n

In [6]:
print(temp)

[1, 0, 3, 7, 2, 0, 0, 0, 3, 3, 1, 2, 0, 1, 0, 0, 0, 3, 0, 1, 1, 1, 0, 5, 3, 3, 1, 3, 4, 1, 0, 3, 4, 1, 1, 2, 1, 1, 1, 1, 4, 1, 2, 2, 1, 2, 1, 2, 2, 4, 7, 1, 3, 6, 0, 1, 0, 0, 0, 0, 6, 2, 7, 0, 4, 3, 0, 0, 1, 1, 0, 6, 2, 2, 1, 0, 0, 0, 6, 3, 1, 1, 0, 2, 4, 0, 0, 0, 3, 6, 1, 5, 2, 2, 7, 1, 3, 0, 2, 5, 1, 1, 5, 2, 1, 0, 1, 3, 0, 0, 6, 4, 4, 0, 2, 2, 0, 1, 3, 1, 4, 2, 2, 4, 3, 3, 0, 0, 2, 2, 2, 0, 0, 5, 3, 0, 0, 1, 1, 3, 1, 1, 1, 3, 0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 2, 0, 3, 3, 1, 4, 1, 3, 0, 1, 3, 2, 1, 2, 2, 1, 5, 0, 3, 7, 1, 1, 1, 3, 3, 0, 1, 0, 3, 1, 1, 0, 1, 1, 2, 3, 4, 0, 4, 0, 1, 3, 0, 1, 2, 0, 1, 5, 1, 1, 0, 2, 4, 0, 3, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 1, 3, 3, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 1, 4, 0, 0, 0, 0, 2, 0, 0, 6, 0, 3, 0, 1, 0, 3, 3, 1, 1, 2, 2, 3, 0, 2, 2, 4, 0, 0, 2, 0, 1, 1, 1, 4, 4, 2, 0, 2, 2, 1, 3, 5, 3, 2, 1, 0, 4, 0, 0, 4, 1, 1, 5, 1, 4, 0, 4, 4, 4, 2, 3, 0, 0, 1, 4, 1, 0, 0, 1, 3, 2, 1, 1, 2, 3, 5, 2, 2, 0, 1, 1, 7, 8, 3, 2, 2, 2, 3, 1, 0, 0, 1, 1, 0, 3, 0, 3, 0, 

In [7]:

power_index = 1
hhn = n >> 2

for i in range(0, hhn):
    temp[ptrf + i], temp[ptrf + hhn + i], temp[ptrf + 2*hhn + i], temp[ptrf + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + i], temp[ptrf + hhn + i], temp[ptrf + 2*hhn + i], temp[ptrf + 3*hhn + i],  omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    temp[ptrg + i], temp[ptrg + hhn + i], temp[ptrg + 2*hhn + i], temp[ptrg + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + i], temp[ptrg + hhn + i], temp[ptrg + 2*hhn + i], temp[ptrg + 3*hhn + i],  omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])

In [8]:
print(temp)

[-4755557, 0, 15052008, -3860830, -4548301, -6283050, 0, -6419528, 2203955, -16501632, 8010713, 207257, 829020, 3387712, 13950029, 10558397, 7248548, -3860834, -9800154, 4149876, -6412442, -11457038, -3860837, 14844755, 3, -14555709, 11109386, 550991, -6068706, -894720, 15395738, -8616392, 207259, -7721673, 10976829, 10089194, 11742157, 10640181, 14501018, -11038607, 550992, 9456422, 8561702, 10640182, -9249165, 1101978, -4211654, 15946728, 4149877, 414514, -16083197, 8561701, -7939932, 4149881, 16579497, 6697561, -9592899, -13117088, -6626783, -6626783, -3571793, 4020484, -9936625, -3309849, 5595588, 5595587, 6353827, -15399659, 1, 10007410, -8698178, 8010718, 1652966, 11534903, 11109386, 13606296, -6834038, 550988, 3173377, -2207870, 8905434, 1101977, 2278650, -13660989, 12078808, 13950029, 1942002, 15052005, 6146575, -2551600, 1309232, -15950642, 4700865, -9800152, 8561707, 10983914, 15395741, -5524807, -8698176, -3309844, -12351759, 2203953, -13109998, 2, 1727663, 10983913, 1539573

In [9]:
n = hhn
hhn = n >> 2
power_index = 4

for j in range(0, 4):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [10]:
print(temp)

[4682066, 10873204, 13273164, 10135471, 2001739, -150128, -3254723, -7673365, -12242520, -15739090, -1655549, -13355031, -6795581, -9852558, -11746565, -905274, 7683850, -3569559, 12388470, 4006604, 8229749, -58935, -14151395, 4673014, 4482536, -6852998, 2343844, 6673172, -7028062, -6252350, 4876281, 13965154, 5936833, -10681511, -8713870, 15673095, -8968259, -8066979, 10128624, -15936683, 4313878, 13099931, 10591110, -11505933, -8465235, -14331747, 7389454, 2999424, 2779421, -10711266, -5966324, -15453640, 11154479, 1937995, -2802119, -5548863, -6051527, -3041226, 481952, -3568106, -7145888, 6562253, 13717904, 11977358, -1183074, -7691370, -1239707, 8635940, 7555631, -12372847, 14625396, 4435501, 1655734, 5403629, -8392102, 7872512, -7983846, -4366661, -11234266, 2829334, -6512873, 2141073, 9105665, 1214228, -1341332, 6813423, -10551091, 10218462, 2326636, 12447819, -2281706, -2005430, -15893553, -11421814, -7635142, 877452, -14406278, -7903601, -14015453, -11932174, 4687558, 14549307

In [11]:
n = hhn
hhn = n >> 2
power_index = 16

for j in range(0, 16):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [12]:
print(temp)

[14107024, 11483618, 8668100, -3127562, 13522219, 8554446, -16372309, 9578703, 7178533, 12898298, -13290777, 13415413, 11931524, 15882357, 9352796, -8919904, 14342938, 9410558, 10083344, 5718679, -9909629, 4412604, 7879135, 6073218, -2761746, 14076856, 16749236, -5332877, 7488266, -13202350, -13315121, 2378935, 2320474, 3205994, -13214394, 5395999, -1741117, -3055577, -9216921, 2349534, 9094164, -4010926, 1362081, -3657279, 4425360, 1447297, -9144691, -12333948, -12042172, -14157691, 14005269, -995569, 6135483, -10511985, 4691203, -15144578, 4619643, 14730423, -11442736, 9255293, 16073200, -9987199, -328907, 15253821, -8103719, -5649977, 338496, 12845438, -14265720, 1001874, 1100702, -11411623, 12452242, 15285169, 15187198, -7357823, 5019550, -10790521, -7841689, -13541755, 357492, -8017942, -901453, 14728664, -11622347, 15270509, 8996190, 8086905, 45686, 8155831, -13804871, 14414146, -3272783, 4946375, 12574033, 1398128, -14115519, -1589524, -10278056, -3275038, 7899596, 11581862, -11

In [13]:
n = hhn
hhn = n >> 2
power_index = 64

for j in range(0, 64):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [14]:
print(temp)


[-14630033, 11707132, -8816911, -8569752, 738733, 10532193, 4350939, 15972044, 1757673, -11538671, -11043794, -8324764, 1461049, 1683481, 16631829, -11587776, 2871108, -15811377, 10302534, 16745473, 7468027, -6280946, 15863675, -7365872, -1320099, -13000482, -11587750, -1849133, 14802379, 5634363, -7795420, 15344248, 3416733, -5812124, -15464743, -6660236, -1848630, -4507505, -14164465, 12909923, 11011821, -332560, -3309305, -14705270, -3298028, -10074172, 13631274, -3510758, 10358025, -509897, -11681315, -7799278, -16334954, 1856628, 14335989, 10629361, 16136841, 10200550, 16455514, -14210176, 8772074, -1077371, 3360551, 7397817, 2933953, 12609738, -10077970, -9117939, -3789186, 16697645, 7347232, 13256169, -3689069, 12740212, -1488653, 2111169, 5679763, 2453171, 5573375, 11582016, -2228160, 15902857, 5162748, -10735889, -9845202, -3092297, 15348768, 9030616, 7403981, 10878394, -15764565, -14171806, 6099349, 11339952, -8352763, 7691061, -14458355, 2215793, 12378233, 8939206, 9697234, 

In [15]:
n = hhn
hhn = n >> 2
power_index = 256

for j in range(0, 256):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
    power_index += 1

In [ ]:
print(temp)

[4007053, -8142000, 9611364, 3104125, 10198393, -15537891, -9706515, -15549392, 14630722, -1886723, 9464306, -15177613, 7104679, 15338530, 5778310, 11173014, -11299894, 15030334, -3003782, 10757774, 5329279, -182356, 16402316, 8322869, -12878783, 12316500, 14362534, 14469690, 10787860, 14045346, -15105920, 15931893, -15184014, 6300379, 10188490, 12362077, 821780, 15340238, 14332149, -4338350, -7467609, 9350086, 13879882, -5265412, -6548180, -11180552, -92707, 4629327, -503362, -5104900, -3972535, -16087777, 16578507, -15786878, -11312685, 12281914, 1074806, 6187367, -15360840, 5545357, 2592085, -14525923, 3755799, 9715998, 11648312, -13068465, -828058, 13984023, 1376820, -15428646, 5457125, -6562043, -4078444, 12237505, -13760853, -9154484, -9345049, 8015647, -15154416, 5652533, -9131409, -5710487, -10761172, 16690428, -3282016, 14295185, -1940484, -14903156, -5344331, -12651402, 13969755, 91565, 13883112, -11206826, -16338597, 4509370, -10239456, -16548922, 9798237, -7292942, -1112943

In [17]:
for num in temp:
    if num > (P[0]>>1):
        print(num)

In [18]:
hhn = 1
n = hhn << 2
power_index = 511

for j in range(0, 256):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [19]:
print(temp)

for i in range(len(temp)):
    if abs(temp[i]) > 3*(P[0]>>1):
        print(i)
        print(temp[i])

[8580542, -2962152, 26353113, 31911330, -30595405, 9747382, 11700834, 2289025, 7030692, -8753757, 33177339, 3485759, -27706141, 8404772, 12365849, 14777882, 11484432, -1538119, 27250026, 5349364, -3678229, 6788022, 10215434, 367422, -5280396, 30170282, 8025529, 27446732, -7891158, -8969486, 24601143, 5436832, -19883405, 6356988, 15466027, 25666548, -7394520, 18756366, 9044590, 31638202, 10496947, -3238334, 25420051, 3657564, -13192112, 14290194, 22284008, 436151, -25668574, 2525950, 30567182, 8511133, 1760858, 18928944, 10719579, 18947570, -2553310, 19086844, 12683817, 29302337, 1537959, -4578934, 22610758, 11632421, 11735812, 7796071, 32625995, 9585214, -15156744, -12258110, 12217482, 6554704, -14756276, 12582620, 15988139, 21419014, -10831285, 12415130, 18617468, 26460139, -8912640, 20572084, 14618344, 28545368, -5830471, 20840875, 5233157, 19847562, -3934413, -11713505, 18712981, 10942522, -9152941, 27288090, 997098, 6357687, -24283083, -9113351, 27126129, 2755400, 5238599, 29758418

In [20]:
hhn = n
n = hhn << 2
power_index = 127

for j in range(0, 64):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [21]:
hhn = n
n = hhn << 2
power_index = 31

for j in range(0, 16):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [22]:
hhn = n
n = hhn << 2
power_index = 7

for j in range(0, 4):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [23]:
hhn = n
n = hhn << 2
power_index = 1

for j in range(0, 1):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [24]:
print(temp)

[-2684025936, -521351085, -150046545, -315031931, -167131704, -173869861, -201302022, -198894532, -15845197, -98225943, -19726314, -96757161, 0, -13162389, -33550337, -26992327, -33550337, -31969184, -33550337, -19004267, -19726314, -25274948, -33550337, -22800078, -25628605, -69194548, 1915090, 4127528, -25889977, -7681896, -33550337, -47776692, 1239962, -10387567, -19726314, -6748068, -53276651, -21964456, -31635247, -9895540, -11804582, -2363219, -29720157, -20404712, -31635247, -20119140, 314450, -31644449, -5902291, -22877376, -20144707, 920250, -27805067, 22411, 0, -38298357, 0, 11280320, 0, 10432982, 1886700, -42366592, -28351764, -25077926, 1239962, -7801707, -33550337, -24390320, -19726314, 618023, 0, -2030568, -5902291, -2340891, 1915090, 7002379, 0, 2524250, 1886700, -5988440, -19726314, -11012351, 0, -12496463, 7660360, 1910814, 0, -3985168, 5745270, -3327723, 314450, 7236949, 628900, -11828260, 5198573, 14003043, 7921732, 3338024, 3830180, -5760006, 1915090, -14770159, 157

In [25]:
Inv2 = 16775169
Inv1024 = ((Inv2)**10) % P[0]

In [26]:
print(Inv1024)

33517573


In [27]:
mu_inv1024 = Mu(Inv1024, P[0])


In [28]:
for i in range(0, 2*n):
    temp[i] = Barrett_mul(temp[i], Inv1024, P[0], mu_inv1024)



In [29]:
print(temp)

[1, 30321793, 27670107, 12109908, 18446738, 26205226, 0, 31390264, 27670107, 25820401, 32810265, 13830211, 0, 30031735, 0, 23268845, 0, 30373773, 0, 28649942, 32810265, 18978438, 0, 23502287, 31330121, 26798908, 26671767, 7081055, 6036057, 28562707, 0, 31013616, 3343139, 3626660, 32810265, 30529459, 32810265, 22258071, 26671767, 20566129, 30590049, 27879857, 19793197, 15968906, 26671767, 20031921, 30863996, 26213062, 32070193, 6268347, 18950684, 10681963, 12914627, 3833410, 0, 24764948, 0, 2107912, 0, 18161445, 17432291, 20927587, 9015177, 5414334, 3343139, 28660882, 0, 22517814, 32810265, 15497976, 0, 32762018, 32070193, 882342, 26671767, 24809187, 0, 30604042, 17432291, 2877384, 32810265, 8344066, 0, 19351321, 6036057, 32569283, 0, 25683085, 12914627, 24471459, 30863996, 22384880, 28177655, 1167953, 9015177, 5157623, 31330121, 7080284, 19793197, 190959, 26671767, 32978925, 20118632, 23740264, 26671767, 18071834, 30863996, 21337844, 0, 15473677, 31691821, 17891927, 6036057, 31846221, 

In [30]:
import random 
for _ in range(1000):
    index = random.randint(0, 1023)
    om = omegas[index]

    if not (Mu(-om, P[0]) == -(mu_omegas[index]-2)):
        print(Mu(-om, P[0]))
        print(-(mu_omegas[index]-2))
        print("error")
        print(index)
        break

    # if not (Mu(-om, P[0]) == -(mu_omegas[index]-2)):
    #     print("error")
    #     print(om)
    #     break


In [31]:
index = 831
print(omegas[index])
print(-omegas[index])
print(-(mu_omegas[index]-2))

print("\n\ntarget:")
print(Mu(-omegas[index], P[0]) == -(mu_omegas[index]-2))

6901341
-6901341
-441739732


target:
True
